In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('./test_score.ipynb'))))

import numpy as np
import pandas as pd

from ylearn.estimator_model.causal_forest import CausalForest
from ylearn.exp_dataset.exp_data import sq_data
from ylearn.utils._common import to_df
from ylearn.estimator_model._naive_forest.utils import grad, grad_coef, inverse_grad

from numpy.linalg import lstsq, inv

from copy import deepcopy


n = 2000
d = 10     
n_x = 1
y, x, v = sq_data(n, d, n_x)

true_te = lambda X: np.hstack([X[:, [0]]**2 + 1, np.ones((X.shape[0], n_x - 1))])
w = deepcopy(v)
v_test = v[:min(100, n)].copy()
v_test[:, 0] = np.linspace(np.percentile(v[:, 0], 1), np.percentile(v[:, 0], 99), min(100, n))
data = to_df(treatment=x, outcome=y, v=v)
test_data = to_df(v=v_test)

outcome = 'outcome'
treatment = 'treatment'
adjustment = data.columns[2:]
from sklearn.preprocessing import OneHotEncoder
oh = OneHotEncoder()
# x = oh.fit_transform(x).toarray().astype(np.float32)

In [2]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

cf = CausalForest(
    x_model=RandomForestClassifier(),
    y_model=RandomForestRegressor(),
    cf_fold=1,
    is_discrete_treatment=True,
    n_jobs=1,
    n_estimators=100,
    random_state=2022,
    min_impurity_decrease=1e-8,
    max_depth=100,
    max_leaf_nodes=100,
    verbose=0
)

In [3]:
cf.fit(data=data, outcome=outcome, treatment=treatment, adjustment=None, covariate=adjustment)

10-08 15:21:02 I ylearn.e.double_ml.py 734 - _fit_1st_stage: fitting x_model RandomForestClassifier
10-08 15:21:03 I ylearn.e.double_ml.py 739 - _fit_1st_stage: fitting y_model RandomForestRegressor


CausalForest(x_model=RandomForestClassifier(), y_model=RandomForestRegressor(), n_estimators=None, min_samples_split=None, min_samples_leaf=None, min_weight_fraction_leaf=None, max_features=None, min_impurity_decrease=None, random_state=2022, ccp_alpha=None, verbose=None, warm_start=None)